### Import Packages

In [1]:
from CustomTransformer import Transformer # this is the transformer.py file

In [2]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np

### Load Dataset

In [3]:
START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

In [4]:
english_file = '../data/english_to_hindi/train.en'
hindi_file = '../data/english_to_hindi/train.hi'

with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(hindi_file, 'r') as file:
    hindi_sentences = file.readlines()

# Remove new-line characters
english_sentences = [sentence.rstrip('\n') for sentence in english_sentences]
hindi_sentences = [sentence.rstrip('\n') for sentence in hindi_sentences]

### Generate English vocabulary

In [5]:
english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@', 

                        'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 
                        'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 
                        'Y', 'Z',

                        '[', '\\', ']', '^', '_', '`', 

                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

### Generate Hindi vocabulary

In [6]:
# hindi_vocab_set = set()
# for sentence in hindi_sentences:
#     for char in sentence:
#         if char not in hindi_vocab_set:
#             if ord(u'\u0900') <= ord(char) <= ord(u'\u097F'):
#                 hindi_vocab_set.add(char)

# hindi_vocabulary1 = list(hindi_vocab_set)

In [7]:
# hindi_vocabulary2 = [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
#                     '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
#                     ':', '<', '=', '>', '?', 'ˌ', '@', 
#                     '१', '२', '३', '४', '५', '६', '७', '८', '९', '०',
#                     '॰', '₹', ';', '।',
                    
#                     'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ',
#                     'ऋ', 'ए', 'ऐ', 'ओ','औ', 'अं', 'अः', 

#                     'क', 'ख', 'ग', 'घ', 'ङ',
#                     'च', 'छ', 'ज', 'झ', 'ञ', 
#                     'ट', 'ठ', 'ड', 'ढ', 'ण',
#                     'त', 'थ', 'द', 'ध', 'न', 
#                     'प', 'फ', 'ब', 'भ', 'म', 

#                     'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 
#                     'त्र', 'ज्ञ', 'श्र', 

#                     # Special characters
#                     'य़', 'ढ़', 'ज़', 'ऱ', 'क़', 'ग़', 'ड़', 'ऴ',
#                     'ऌ', 'ख़', 'ॐ', 

#                     # Matras
#                     'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'ॅ', 'ॆ',
#                     'े', 'ै', 'ॉ', 'ॊ', 'ो', 'ौ',  '्', 'ॅ्']

# hindi_vocabulary = list(set(hindi_vocabulary1+hindi_vocabulary2))
# hindi_vocabulary.insert(0, START_TOKEN)
# hindi_vocabulary.append(PADDING_TOKEN)
# hindi_vocabulary.append(END_TOKEN)
# print(hindi_vocabulary)

In [8]:
hindi_vocabulary = ['<START>', 'ॽ', 'ॡ', 'श्र', '>', '+', 'ढ', '$', 'ॄ', 'य़', '७', '॒', 'ज', '0', '॓', 'श', '!', '"', 'ी', 'ो', 'झ', 'ॱ', '5', 'स', 'े', 'ॐ', 'य', 'ऌ', 'ग़', 'ई', 'ं', 'त', 'ॅ', 'क़', 'च', '%', '़', 'ऐ', 'ू', 'ऱ', "'", ':', '(', 'घ', 'ऍ', 'ट', 'ऊ', '/', 'ण', '*', 'ः', 'र', 'प', '4', 'त्र', 'ग', 'ˌ', 'ज्ञ', '6', 'ठ', 'ा', 'ञ', '7', ',', 'ॅ्', '9', 'ॆ', '-', '<', 'ऺ', '९', '्', 'भ', 'ङ', 'ड़', '॥', 'ज़', '२', '1', '&', 'ध', 'फ', 'ळ', 'फ़', ';', 'ख', 'ऩ', 'व', 'ि', 'ै', '=', 'ए', 'ड', '५', 'ख़', 'ऎ', ' ', '#', 'अः', 'ु', 'ऒ', 'छ', 'द', '@', 'ढ़', '।', 'उ', 'ऽ', 'ॠ', 'म', '१', 'ल', '॰', 'ँ', '?', '६', 'ौ', 'ॢ', 'ॹ', 'न', 'ऑ', '3', 'ओ', '॑', 'ष', '३', 'ॉ', 'ऋ', 'ॲ', 'इ', '॔', 'ॣ', '८', '.', '०', '8', 'अं', 'थ', ')', '४', '2', 'औ', 'ब', 'ऴ', 'ृ', 'क', 'अ', 'ॊ', '₹', 'ह', 'आ', '<PADDING>', '<END>']

In [9]:
index_to_hindi = {k:v for k,v in enumerate(hindi_vocabulary)}
hindi_to_index = {v:k for k,v in enumerate(hindi_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [10]:
# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
hindi_sentences = hindi_sentences[:TOTAL_SENTENCES]

In [11]:
english_sentences[:10]

["However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles",
 'Whosoever desires the reward of the world, with Allah is the reward of the world and of the Everlasting Life. Allah is the Hearer, the Seer.',
 'The value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.',
 'Mithali To Anchor Indian Team Against Australia in ODIs',
 'After the assent of the Honble President on 8thSeptember, 2016, the 101thConstitutional Amendment Act, 2016 came into existence',
 'The court has fixed a hearing for February 12',
 'Please select the position where the track should be split.',
 'As per police, armys 22RR, special operation Group (SOG) of police and the Central Reserve Police Force (CRPF) cordoned the village and launched search operation in the area.',
 'Jharkhand chief minister Hemant Soren',
 'Arvind Kumar, SHO of the sector 55/56 police sta

In [12]:
hindi_sentences[:10]

['आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।',
 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है',
 'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले में उनकी संख्या अन्य जीव समूहों से ज़्यादा है।',
 'आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को',
 '8 सितम्\u200dबर, 2016 को माननीय राष्\u200dट्रपति की स्\u200dवीकृति मिलने के बाद 101वां संविधान संशोधन अधिनियम, 2016 अस्तित्\u200dव में आया',
 'अदालत ने इस मामले में आगे की सुनवाई के लिए एक फरवरी की तारीख़ तय की',
 'जहाँ पर ट्रैक को विभाजित किया जाना है, कृपया वह स्थान चुनें.',
 'इसके तुरंत बाद सेना की 22 राष्ट्रीय राइफल्स (आरआर), सीआरपीएफ और पुलिस के स्पेशल ऑपरेशन ग्रुप (एसओजी) के जवानों द्वारा इलाके की घेराबंदी कर तलाशी अभियान चलाया।',
 'झारखंड के मुख्यमंत्री हेमंत सोरेन (फोटोः पीटीआई)',
 'सेक्टर 55/56

In [13]:
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Hindi: {np.percentile([len(x) for x in hindi_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )

97th percentile length Hindi: 258.0
97th percentile length English: 267.0


In [14]:
# Model Parameters
d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 300
hindi_vocab_size = len(hindi_vocabulary)

In [15]:
def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(hindi_sentences)):
    hindi_sentence, english_sentence = hindi_sentences[index], english_sentences[index]
    if is_valid_length(hindi_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(hindi_sentence, hindi_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(hindi_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 200000
Number of valid sentences: 165201


In [16]:
hindi_sentences = [hindi_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [17]:
english_sentences[:3]

["However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles",
 'Whosoever desires the reward of the world, with Allah is the reward of the world and of the Everlasting Life. Allah is the Hearer, the Seer.',
 'The value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.']

In [18]:
hindi_sentences[:3]

['आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।',
 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है',
 'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले में उनकी संख्या अन्य जीव समूहों से ज़्यादा है।']

# Initialize Transformer

In [19]:
transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          hindi_vocab_size,
                          english_to_index,
                          hindi_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

In [20]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(97, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [21]:
class TextDataset(Dataset):

    def __init__(self, english_sentences, hindi_sentences):
        self.english_sentences = english_sentences
        self.hindi_sentences = hindi_sentences

    def __len__(self):
        return len(self.english_sentences) #+ len(self.hindi_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.hindi_sentences[idx]

In [22]:
dataset = TextDataset(english_sentences, hindi_sentences)

In [23]:
len(dataset)

165201

In [24]:
dataset[0]

("However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles",
 'आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।')

In [25]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=hindi_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [26]:
NEG_INFTY = -1e9

def create_masks(eng_batch, hn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, hn_sentence_length = len(eng_batch[idx]), len(hn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      hn_chars_to_padding_mask = np.arange(hn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, hn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, hn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, hn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

Modify mask such that the padding tokens cannot look ahead.
In Encoder, tokens before it should be -1e9 while tokens after it should be -inf.
 

Note the target mask starts with 2 rows of non masked items: https://github.com/SamLynnEvans/Transformer/blob/master/Beam.py#L55


In [27]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, hn_batch = batch

        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, hn_batch)

        optim.zero_grad()
        hn_predictions = transformer(eng_batch,
                                     hn_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        
        labels = transformer.decoder.sentence_embedding.batch_tokenize(hn_batch, start_token=False, end_token=True)
        loss = criterian(
            hn_predictions.view(-1, hindi_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == hindi_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hn_batch[0]}")
            hn_sentence_predicted = torch.argmax(hn_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in hn_sentence_predicted:
              if idx == hindi_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_hindi[idx.item()]
            print(f"Hindi Prediction: {predicted_sentence}")


            transformer.eval()
            hn_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hn_sentence)
                predictions = transformer(eng_sentence,
                                          hn_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_hindi[next_token_index]
                hn_sentence = (hn_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break
            
            print(f"Evaluation translation (should we go to the mall?) : {hn_sentence}")
            print("-------------------------------------------")

Epoch 0
Iteration 0 : 5.948587417602539
English: However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles
Hindi Translation: आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।
Hindi Prediction: ळळळ३ळल३ळढ़.#३ळळळळळ३३ळळळळळळऍळ.ळळळशऊ३ळळळळ.ढ़....ळळ.ळ.ऊ.....ॐॐळळ३ै...ळळ.ळ्.ळ..ळळ?.ऍळ॓ळ.ळ#ळ..ळ#.ॐअं.ऱ...ळ.्ळ.्््.ळ.ऍ्ळळऎळ५..ळळऍअंळऍआ#ळऱ््9आआळळ.%आऍआआळगॢआ#ॢळॢ.ॢॢ#ळ'ळज़घळआआळगआअं'ळळ'ॢळळआळळलळ#ळळ9ऍळळऊऍइळऊ9३घ9ै9999ॢ३6--ळळळघळघ9.ॢळ<PADDING>9ळ9%9999घ%%ऊऊऊआआ३ळऊ'9ऊँआआआआ4तत ऊत.%.आ ॑श्र॑ 9     आदआ आआआआआआआआ्ऊऍआआआञआ  ऍऍ३ददऊदञघ्घघ'घ9ग9
Evaluation translation (should we go to the mall?) : ('                           ळळळ                                                                                                                                                                                                                                   

## Inference

In [28]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  hn_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hn_sentence)
    predictions = transformer(eng_sentence,
                              hn_sentence,
                              encoder_self_attention_mask.to(device), 
                              decoder_self_attention_mask.to(device), 
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_hindi[next_token_index]
    hn_sentence = (hn_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return hn_sentence[0]

In [35]:
translation = translate("hello")
print(translation)

क्य है<END>


In [31]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in transformer.state_dict():
    print(param_tensor, "\t", transformer.state_dict()[param_tensor].size())

Model's state_dict:
encoder.sentence_embedding.embedding.weight 	 torch.Size([97, 512])
encoder.layers.0.attention.qkv_layer.weight 	 torch.Size([1536, 512])
encoder.layers.0.attention.qkv_layer.bias 	 torch.Size([1536])
encoder.layers.0.attention.linear_layer.weight 	 torch.Size([512, 512])
encoder.layers.0.attention.linear_layer.bias 	 torch.Size([512])
encoder.layers.0.norm1.gamma 	 torch.Size([512])
encoder.layers.0.norm1.beta 	 torch.Size([512])
encoder.layers.0.ffn.linear1.weight 	 torch.Size([2048, 512])
encoder.layers.0.ffn.linear1.bias 	 torch.Size([2048])
encoder.layers.0.ffn.linear2.weight 	 torch.Size([512, 2048])
encoder.layers.0.ffn.linear2.bias 	 torch.Size([512])
encoder.layers.0.norm2.gamma 	 torch.Size([512])
encoder.layers.0.norm2.beta 	 torch.Size([512])
decoder.sentence_embedding.embedding.weight 	 torch.Size([153, 512])
decoder.layers.0.self_attention.qkv_layer.weight 	 torch.Size([1536, 512])
decoder.layers.0.self_attention.qkv_layer.bias 	 torch.Size([1536])
dec

In [33]:
# Save Model
PATH = "../saved_models/Trained_Transformer_1"
torch.save(transformer.state_dict(), PATH)

In [34]:
# Load Model
PATH = "../saved_models/Trained_Transformer_1"

model = transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          hindi_vocab_size,
                          english_to_index,
                          hindi_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>